# Introduction

NoSQL databases are a popular alternative to traditional relational databases for handling big data and it offers high scalability, high availability and high flexibility. The course will teach you how to work with NoSQL databases and how to choose the right NoSQL database for your specific needs. 

## Structure

- [Relational databases and tabular data](#RDBMs)
- [SQLite](#SQLite)
- [MySQL](#MySQL)
- [PostgreSQL](#PostgreSQL)
- [Intro to NoSQL](#NoSQL)
- [Exercise](#Exercise)


<a name="RDBMs"></a>
## Relational databases and tabular data

Before jumping straight on NoSQL DBs let's take a step back with relational databases and tabular data. The most common format you probably encountered in programming is Comma Separated Values (CSV). CSV is a plain text file format used to store tabular data, with each row of the table represented as a separate line in the file and each column separated by a comma or other delimiter.



In [ ]:
import pandas as pd

df = pd.read_csv("data/Chap1/iris.csv")
df.head(5)

In [ ]:
!pip install pandas

In [ ]:
print(df["sepal.length"])
print(df.at[0,"sepal.length"])
print(df.loc[0])
df[(df["sepal.length"] > 3) & (df["petal.length"] > 1.2)]

In [ ]:
# Iterate over all db

for row in df.iterrows():
    print(row)
    break

Yet tabular data are impractical for huge amount of data 

Relational databases are a type of database management system that organize data into one or more tables or relations, with each table consisting of a set of rows and columns. Each row represents a unique record or entity, while each column represents a particular attribute or characteristic of that entity.

In a relational database, the tables are related to each other through common fields or keys. For example, if you have a customer table and an orders table, you could link them through a customer ID field, which would allow you to associate specific orders with specific customers.

Relational databases use a structured query language (SQL) to retrieve and manipulate data. SQL allows users to create, update, delete, and retrieve data from one or more tables based on specific criteria or conditions.

Relational databases offer several advantages, including:

- Data consistency: Data is consistent and standardized across an organization.

- Scalability: Relational databases can handle large amounts of data and can easily scale as data volumes increase (Data not in your RAM).

- Flexibility: Changes to a database structure can be made relatively easily and without disrupting existing applications or data.

- Security: Relational databases offer built-in security features, such as user authentication and access control, to protect sensitive data.


<a name="SQLite"></a>
## SQLite

I'll do a short introduction on sqlite3. I think it's a good entry point to SQL since its local and you can still monitor your data visually using [DB Browser](https://sqlitebrowser.org/).

"SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine." [(Official website)](https://www.sqlite.org/index.html). Sqlite3 is a library that provides an api that allows python 3 to interact with your sqlite DB. Incompatibility with python 2 so beware.

You can use [DB browser](https://sqlitebrowser.org/) to check if it worked without going through Python. Sqlite3+DB browser is an easy entry point to SQL (still able to see your data, no server involved). Here is an example of basic operations of SQLite on the iris database. Starting with DB already existing:

In [ ]:
import sqlite3

#  Establishes a connection to an SQLite database file. The only argument is the path (If the file does not exist and empty file is created)
conn = sqlite3.connect('data/Chap1/iris.db')

# A cursor in SQL databases is a database object that allows you to retrieve and manipulate a set of data rows returned by a SQL statement, one row at a time.
c = conn.cursor()

# READ query (SQL language), Look at all the iris with sepal length > 5.
c.execute("""SELECT * FROM iris WHERE "petal.length" > 5""")

#%% Two methods to go through your result

# All in RAM
docs = c.fetchall()
print(docs)

# Iteration

c.execute("""SELECT * FROM iris WHERE "petal.length" > 5""")

for i in c:
    print(i)
    break


In [ ]:
# UPDATE

c.execute("""UPDATE iris
             SET "sepal.length" = 3
             WHERE variety == "Versicolor" """)

conn.commit()

In [ ]:
c.execute("""UPDATE iris
             SET "sepal.length" = random()+1
             WHERE variety == "Versicolor" """)

conn.commit()

In [ ]:
# DELETE

c.execute("""DELETE FROM iris WHERE "sepal.length" > 5""")

#conn.commit() warning it will alter your db 

Now if the Database does not exist we can create it the following way

In [ ]:
import tqdm
import sqlite3
import pandas as pd

df = pd.read_csv('data/Chap1/iris.csv')

conn = sqlite3.connect('data/Chap1/iris_manual.db')
c = conn.cursor()

# Simple function to create table

def create_table_iris():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS iris(id INT PRIMARY KEY, "sepal.length" FLOAT, "sepal.width" FLOAT,
                     "petal.length" FLOAT, "petal.width" FLOAT, variety TEXT)""")
        # When an index is created on a column, the database system creates a data structure that contains a sorted copy of the column's values. 
        # This allows the database system to perform data retrieval operations more quickly than if it had to scan the entire table every time a query was executed.
        c.execute("CREATE INDEX fast_id ON id(id)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_iris()



In [ ]:
# One by one

query = """INSERT INTO iris(id, "sepal.length", "sepal.width", "petal.length", "petal.width", variety) VALUES (?,?,?,?,?,?)"""

for row in tqdm.tqdm(df.iterrows()):
    id_ = row[0]
    sepal_length = row[1]["sepal.length"]
    sepal_width = row[1]["sepal.width"]
    petal_length = row[1]["petal.length"]
    petal_width = row[1]["petal.width"]
    variety = row[1]["variety"]
    values = (id_,sepal_length,sepal_width,petal_length, petal_width,variety)
    c.execute(query, values)
    conn.commit()

In [ ]:
import sqlite3
import tqdm

conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

# Simple function to create table

def create_table_benchmark():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS test(id INT PRIMARY KEY)""")
        # When an index is created on a column, the database system creates a data structure that contains a sorted copy of the column's values. 
        # This allows the database system to perform data retrieval operations more quickly than if it had to scan the entire table every time a query was executed.
        c.execute("CREATE INDEX fast_id ON id(id)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_benchmark()


query = """INSERT INTO test(id) VALUES (?)"""

for i in tqdm.tqdm(range(0,100,1)):
    values = (i,)
    c.execute(query, values)
    conn.commit()

In [ ]:
import sqlite3
import tqdm

conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

# Simple function to create table

def create_table_benchmark():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS test2(id INT PRIMARY KEY)""")
        # When an index is created on a column, the database system creates a data structure that contains a sorted copy of the column's values. 
        # This allows the database system to perform data retrieval operations more quickly than if it had to scan the entire table every time a query was executed.
        c.execute("CREATE INDEX fast_id ON id(id)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_benchmark()


query = """INSERT INTO test2(id) VALUES (?)"""

for i in tqdm.tqdm(range(0,1000000,1)):
    values = (i,)
    c.execute(query, values)
conn.commit()

In [ ]:
import sqlite3
import tqdm

conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

def create_table_benchmark():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS test3(id INT PRIMARY KEY)""")
        # When an index is created on a column, the database system creates a data structure that contains a sorted copy of the column's values. 
        # This allows the database system to perform data retrieval operations more quickly than if it had to scan the entire table every time a query was executed.
        c.execute("CREATE INDEX fast_id ON id(id)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_benchmark()

query = """INSERT INTO test3(id) VALUES (?)"""


list_of_insertion = []
for i in tqdm.tqdm(range(0,1000000,2)):
    values = (i,)
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()

Here are some desc stats you can do using SQL language

In [ ]:
# Aggreagte function

import sqlite3

conn = sqlite3.connect('data/Chap1/iris_manual.db')
c = conn.cursor()

c.execute("""SELECT "petal.length", variety, AVG("petal.width"), SUM("petal.width"), COUNT("petal.width"), MIN("petal.width"), MAX("petal.width")
             FROM iris 
             GROUP BY variety
             """)

docs = c.fetchall()
print(docs)

Data won't be this clean in your day-to-day job. Let us take an example of a research article each observation can have different length of authors/references/...

In [ ]:
# Init variables
authors = ["Auteur1","Auteur2","Auteur3"]
title = "This is paper 1"
affiliations = ["University of Mannheim","University of Strasbourg"]
ref = ["This is ref 1","This is ref 2","This is ref 3"]

In [ ]:
# Collapse list

authors = "\n".join(authors)
aff = "\n".join(affiliations)
ref = "\n".join(ref)
print(authors)

In [ ]:
# Create the DB
import sqlite3

# connect to your db, if it does not exists it creates it.
conn = sqlite3.connect('data/Chap1/tuts.db')

c = conn.cursor()

# Simple function to create table

def create_table_test():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS test(id INT PRIMARY KEY, authors TEXT,
                    title TEXT,affiliations TEXT, ref TEXT)""")
        # Index to do query faster
        c.execute("CREATE INDEX fast_id ON id(id)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_test()


In [ ]:
# SQL can be finicky when trying to insert using python, big difference between " and '
query = """INSERT INTO test(id, authors,title,affiliations, ref) VALUES (?,?,?,?,?)"""
values = (3,authors,title,aff, ref)
c.execute(query, values)
conn.commit()

Now that we have a DB with multiple tables (benchmark) I think its time to introduce a fundamental concept in SQL: **Join**

**Join:** A join is an operation that combines rows from two or more tables based on a related column between them. It allows you to retrieve data from multiple tables simultaneously, based on a common column or set of columns. There is multiple types of join Left Join, Right Join, Full Join, Inner Join.

Since the goal of the course is NoSQL we will go more into details of the Joins in Chapter III. Here is a small example of left join.

In [ ]:
import sqlite3

conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

c.execute("""SELECT test.id as id_table1, test2.id as id_table2
             FROM test 
             LEFT JOIN test2 
             ON test.id=test2.id
             """)

docs = c.fetchall()
print(docs)

Consider a scenario where you've performed a left join operation, combining data from multiple tables, and you now want to work with this combined dataset without saving it permanently. You might find yourself repeatedly executing this join query whenever you need to perform more advanced operations, such as using aggregate functions. Alternatively, you may wish to streamline your script by creating a 'shortcut' that automatically executes this join query when needed, improving readability and maintainability.

This is where the concept of a **'View'** becomes valuable. A view in SQL allows you to define a virtual table based on a query's result set, without physically storing the data. Essentially, a view acts as a saved query that can be referenced like a table. It provides a convenient way to encapsulate complex queries and present data in a structured format.

It's important to note that while using a view enhances readability and organization of your SQL code, the underlying query defined in the view is still executed each time the view is accessed. Therefore, there's no performance gain in terms of execution time; rather, the benefit lies in improved code readability and maintainability."

In [ ]:
import sqlite3

conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

# Create the view
c.execute("""CREATE VIEW test_join_view AS
             SELECT test.id as id_table1, test2.id as id_table2
             FROM test 
             LEFT JOIN test2 
             ON test.id=test2.id""")

# Commit the transaction
conn.commit()

# Close the connection
conn.close()

In [ ]:
# Connect to the database
conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

# Query the view
c.execute("""SELECT * FROM test_join_view
             WHERE id_table1 > 50""")
docs = c.fetchall()
print(docs)

# Close the connection
conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data/Chap1/benchmark.db')
c = conn.cursor()

# Drop the view
c.execute("DROP VIEW IF EXISTS test_join_view")

# Commit the transaction
conn.commit()

# Close the connection
conn.close()

[Here]( https://www.sqltutorial.org/sql-cheat-sheet/) are some cheat sheet for SQL querys.

SQLite is really useful to learn the language but chances are you will work with an online server and will need to use things like MySQL, Oracle, PostgreSQL,...
The language is always (almost) the same but the way you will connect and run query might change.

<a name="MySQL"></a>
## MySQL

MySQL is an open-source relational database management system (RDBMS) that is based on the structure query language (SQL). It is developed, distributed, and supported by Oracle Corporation. It provides a powerful, scalable, and reliable platform for managing and manipulating large sets of data. MySQL is known for its ease of use, performance, and robustness, making it a widely adopted solution for various types of database-driven applications ranging from small-scale projects to enterprise-level systems.

Let's just do a simple example of MySQL:

From here https://dev.mysql.com/downloads/ you need to DL the community server, the workbench and the python connector. Once they are all installed and the server is up and running you should be able to run the next code

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

print(mydb) 

c = mydb.cursor()

c.execute("CREATE DATABASE mydatabase")


If a database already exist you can connect directly to it

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="mydatabase"
)

c = mydb.cursor()

c.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")


You can now look at the DB in the workbench and execute query as seen in SQLite

For the rest of this section I will show you a more concrete examples. The data is from this [PowerBI tutorial](https://www.youtube.com/watch?v=e6QD8lP-m6E) (I really recommend it just to put PowerBI in your CV)

In [ ]:
import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="m1_ds2e"
)

c = mydb.cursor()

c.execute("SELECT * FROM sample_superstore")

docs = c.fetchall()

columns = [description[0] for description in c.description]
df = pd.DataFrame(docs, columns=columns)

In [ ]:
df.head()

In [ ]:
df.columns

I think this data is perfect to introduce more advanced concepts in SQL.

**Group by**: The GROUP BY clause in SQL is used to group rows that have the same values into summary rows, typically to perform aggregate functions on these groups. When you use GROUP BY, you are essentially creating groups of rows based on the values of one or more columns, and then applying aggregate functions to each group.

**Window function**: Window functions perform calculations across a set of rows related to the current row within a query result set. They allow you to perform calculations and aggregations without grouping the result set into single rows, providing a more flexible and powerful way to analyze data.



In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute( """
SELECT Region, AVG(Profit) AS avg_profit
FROM sample_superstore
GROUP BY Region
""")

docs = c.fetchall()
print(docs)

In [ ]:
import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="m1_ds2e"
)

c = mydb.cursor()

c.execute( """
SELECT Region, Profit,
RANK() OVER (PARTITION BY Region ORDER BY Profit DESC) AS profit_rank
FROM sample_superstore
""")

docs = c.fetchall()
df = pd.DataFrame(docs, columns=["Region","Profit","Profit_Rank"])

In [ ]:
print(df[df["Region"]=="West"])

**Case statement**: 
The CASE statement in SQL is a powerful and flexible tool for performing conditional logic within queries. It allows you to evaluate one or more conditions and return a result based on the outcome of those conditions. The basic syntax of the CASE statement is as follows:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute( """
SELECT `Product Name`, Discount,
    CASE 
        WHEN Discount >= 0.05 THEN 'High discount'
        ELSE 'Low discount'
    END AS Discount_Category
FROM sample_superstore
""")

docs = c.fetchall()
print(docs)

The last thing I want to show you is **Trigger Event**. A trigger event, in the context of databases, refers to an action or operation that occurs within the database system and triggers the execution of a trigger. Triggers are special types of stored procedures that are automatically invoked in response to predefined events or actions, such as INSERT, UPDATE, DELETE operations on a table, or database-specific events like startup or shutdown. To demonstrate this we will create a new table which is an aggregate of profit and once a row is added in the original table then a trigger will launch to add a new row with the sum of porift and date.

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute( """
CREATE TABLE profit_summary (
    Total_Profit DECIMAL(10, 2),
    Date_insert DATE DEFAULT (CURRENT_DATE)
)
""")



In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute( """
CREATE TRIGGER update_profit_agg
AFTER INSERT ON Profit
FOR EACH ROW
BEGIN
    DECLARE total_profit DECIMAL(10, 2);
    SELECT SUM(Profit) INTO total_profit
    FROM Profit;
    INSERT INTO profit_summary (Total_Profit) VALUES (total_profit);
END;
""")

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute("""
INSERT INTO Profit (Profit) VALUES (25000)
""")
mydb.commit()

<a name="PostgreSQL"></a>
## PostgreSQL

https://www.enterprisedb.com/downloads/postgres-postgresql-downloads
PostgreSQL, often referred to as "Postgres," is an advanced open-source relational database management system (RDBMS) known for its robustness, reliability, and feature-rich capabilities. It is widely recognized for its adherence to SQL standards, extensibility, and support for advanced data types and features. For this section we will work on a dataset from UCI https://archive.ics.uci.edu/dataset/222/bank+marketing

In [ ]:
import pandas as pd

df = pd.read_csv("data/Chap1/bank-full.csv")
df.columns

In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="new_db", user="postgres", password="ROOT", host="localhost",  port="5432" )


In [ ]:
# Create a cursor object
cur = conn.cursor()

# Define the table schema
table_name = "bank_full"
columns = [
    "age INTEGER",
    "job VARCHAR(100)",
    "marital VARCHAR(20)",
    "education VARCHAR(100)",
    "default_ VARCHAR(10)",
    "balance NUMERIC",
    "housing VARCHAR(10)",
    "loan VARCHAR(10)",
    "contact VARCHAR(100)",
    "day INTEGER",
    "month VARCHAR(20)",
    "duration INTEGER",
    "campaign INTEGER",
    "pdays INTEGER",
    "previous INTEGER",
    "poutcome VARCHAR(100)",
    "y VARCHAR(10)"
]

# Construct the CREATE TABLE query
create_table_query = (
    "CREATE TABLE IF NOT EXISTS " + table_name + " (" +
    ", ".join(columns) +
    ")"
)

# Execute the CREATE TABLE query
cur.execute(create_table_query)

# Commit the transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()

<a name="NoSQL"></a>
## NoSQL

A csv (or RDBMS) works but is it really efficient ? Imagine a scenario with even more nested data (each author has an aff, an aff can have a list a members, each members has an age, ...).You could create different tables but each time you will need to join information of this different tables. That's where NoSQL comes in to save the day.


Modifying the architecture in SQL is clunky. A lot of time is invested designing the data model because changes will impact all or most of the layers in the application.

NoSQL is:
- Meant for unstructured data.
- Designed to manage lots of traffic and data.
- Has no predefined schema.
- Easy to query.

Because of the context (Big Data), Unstructured data has become more and more present hence the interest in NoSQL. Indeed JSON (more on that in chapter I) data is the commonly used data format in Javascript and Javascript is present everywhere in almost every web page (dynamic vs static).

The biggest advantage of NoSQL is that you get to skip the "understand your data and usage patterns" steps. But is it really an advantage ?

NoSQL cons:
- Don't really know what you are working with
- Multiplicity of different DB (Meaning different "language" for some)
- Flexible but can become messy so watch out

IMO: Use SQL when you have a Static data flow or the change in the data is not regular and if the data fits in a row-column format. It's always better to have a good structure even if it asks to put a lot of work before hand. If you need flexibility or the data is really nested using Nosql DBs reduces the need for joins and lookups, making your queries faster.


<p style="text-align: center;"><b>Proportion of DBs usage</b></p>

![test](./img/DB_piechart.png "Title")

Nothing new but we are surrounded by a lot of data, information flows faster than ever (i.e: big data). But what exactly is the type of data we get in huge quantity ? Text, Networks, Sound, Image, ...

## What is NoSQL ? SQL vs NoSQL

NoSQL stands for "Not only SQL", every database that does not use SQL (Structured Query Language) is called NoSQL even though every NoSQL DB does not work the same way. The difference between SQL and NoSQL databases is really just a comparison of relational vs. non-relational databases. Deciding when to use SQL vs. NoSQL depends on the kind of information you’re storing and the best way to store it. Both types store data, they just store data differently.
Imagine you have a list of variables for research papers. authors, affiliations, title, references. How do you store it in a csv ? 

The popularity of DBs using SQL is still high (3/4). In your opinion why ? 

<p style="text-align: center;"><b>Non exhaustive list of companies using NoSQL</b></p>

<table><tr>
<td> <img src="./img/Google.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="./img/Amazon logo.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="./img/Facebook.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="./img/Mozilla.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="./img/netflix.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="./img/Twitter.png" alt="Drawing" style="width: 150px;"/> </td>
</tr></table>

Think before using one or the other.

Before jumping into NoSQL directly we will take a look at what a unstructured data format looks like.

<a name="Exercise"></a>
## Exercise

## SQLITE

**1**: Read all the data from bc.csv, how many rows is there ? Columns ? 

**2**: Insert the bc.csv in the sqlite format.

**3**: In the same db create a new table called tomatch with two variables id_ and rdm_float. 

**4**: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.

**5**: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

**6**: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

**7**: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups


**8**: Change the name of the column "diagnosis" to "label"

**9**: Add a column named "day" of type DATE.

**10**: Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)

**11**: create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.

**12**: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2. 

## MySQL

**1**: How many orders were placed in total?

**2**: What is the total sales amount?

**3**: What is the average profit per order?

**4**: Which customer has placed the most orders?

**5**: What is the total profit for each product category?

**6**: What is the average discount percentage?

**7**: Which region has the highest total sales?

**8**: How many orders were shipped using each shipping mode?

**9**: What is the total quantity ordered for each product sub-category?

**10**: Which product has the highest profit margin?

**11**: What is the total sales amount for each year?

**12**: Which customer segment has the highest average sales?

## PostgreSQL

**1**: What is the average age of the customers?

**2**: How many customers have defaulted on loans?

**3**: What is the maximum balance in the account?

**4**: How many customers have a housing loan?

**5**: What is the education level of the youngest customer?

**6**: What is the marital status of the oldest customer?

**7**: What is the distribution of the months in which customers were last contacted?

**8**: What is the average duration of contact for customers who have a personal loan?

**9**: How many customers were previously contacted, and what was the average duration of their last contact?

**10**: What is the distribution of job types among customers?

**11**: How many customers have been contacted more than once in the current campaign?

**12**: What is the success rate of previous marketing campaigns (poutcome) among customers